In [1]:
import pandas as pd 
import numpy as np

### In Defense of VaR: 

People like to criticize value at risk, and there has been a big push from certain quarters to favor expected shortfall (aka ES, conditional VaR, or cVaR) over VaR. I think this push is misguided (VaR is not as bad as they make it out to be and ES has its own shortcomings). 
 
For this article I wanted to look at how the ratio of ES over VaR changes over time. I suspect most practitioners expect this ratio to increase in stress environments (ES is supposed to tell us a lot about the shape of tails; we associate stress environments with tail risk; therefore ES should "light up" more in a stress environment than VaR). I wonder if the opposite is not true.
 
To start, we could look at  realized VaR and ES for the S&P 500 in each calendar year from 1995 to present (you'd have at least three big market declines: tech bubble bursting in 1999, financial crisis in 2008, and current market). Depending on initial results, we could look at other indices (other countries, other asset classes) and/or hypothetical long/short portfolios.

## What is Value at Risk (VaR)

A measure called value at Risk (or $\operatorname{VaR}$ ) was created to answer the question "how bad can things get realistically." It's now widely used and, in some cases, required by financial regulators to be computed and acted on.

VaR is simply a percentile of a probability distribution of a financial quantity of interest. The quantity is usually either:
- _P&L_ (profit&loss, i.e. change in wealth from the current wealth); or
- _Rate of return_ or just _return_ of wealth; that's ending wealth divided by beginning wealth, all minus one;
- _Log-return_, which is the logarithm of one plus the rate of return.

Note that all of these quantities can in general take on both positive and negative values. Value at Risk is concerned with how much can be lost, so the convention is to reverse the sign in a way we'll describe below, giving bigger positive numbers for bigger losses.

Let $X$ be the random variable giving the future value of whichever one of the quantities above is being analyzed, and let $F_t(x)$ be the cumulative distribution function (cdf) of $X$'s value at some time $t$ in the future. The Value at Risk (VaR) over time $t>0$ with probability $0\leq p\leq 1$ is:
$$VaR_t(p)=-\inf\{x\mid F_t(x)=Pr(X\le x)\ge 1-p\}$$

If $X$'s cdf $F_t$ is continuous, then the calculation simplifies to:
$$VaR_t(p)=-F_t^{-1}(1-p);\hspace{2em}VaR_t(p)=x \iff \int_{-\infty}^{-x}f_t(y)dy=1-p$$
where $f_t(y)$ is the probability density function of the distribution.

In words, we say that $VaR_t(p)$ is the **`t-year p Value at Risk`**. $t$ could also be denominated in other time units like days. The time argument $t$ might be left implicit if it is specified elsewhere.


### An example of VaR?

For example, if a bank wants to estimate how bad things can get in its trading operations, the random variable of interest might be $\Delta w$, the change (P&L) in its trading capital $w$ by the end of tomorrow's trading day. The one-day $99\%$ VaR for its trading operations would be the (positive) loss amount that was expected to exceed $-\Delta w$, $99$ days out of $100$. So if the one-day $99\%$ VaR is $\$50,000,000$, then the bank expects to lose less than $\$50,000,000$ on all but one out of $100$ trading days.

### realized VaR and ES for the S&P 500 in each calendar year from 1995 to present

In [ ]:
# SP500
